In [ ]:
# i'm installing opencv library
!apt update
!apt install -y libopencv-dev build-essential


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,078 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,410 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Package

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%%writefile resize_image.cpp
#include <opencv2/opencv.hpp>
#include <iostream>

int main() {
    std::string input_path = "/content/drive/MyDrive/G178_2 -1080.BMP";

    cv::Mat src = cv::imread(input_path);
    if (src.empty()) {
        std::cerr << "Error: Could not load image!" << std::endl;
        return -1;
    }

    cv::Size targetSize(src.cols / 2, src.rows / 2);

    cv::Mat resized_nearest, resized_linear, resized_cubic;

    cv::resize(src, resized_nearest, targetSize, 0, 0, cv::INTER_NEAREST);
    cv::resize(src, resized_linear, targetSize, 0, 0, cv::INTER_LINEAR);
    cv::resize(src, resized_cubic, targetSize, 0, 0, cv::INTER_CUBIC);

    cv::imwrite("/content/drive/MyDrive/output_nearest.bmp", resized_nearest);
    cv::imwrite("/content/drive/MyDrive/output_linear.bmp", resized_linear);
    cv::imwrite("/content/drive/MyDrive/output_cubic.bmp", resized_cubic);

    std::cout << "Done resizing and saving images." << std::endl;
    return 0;
}


Writing resize_image.cpp


In [4]:
!g++ resize_image.cpp -o resize_image `pkg-config --cflags --libs opencv4`

In [5]:
!./resize_image

Done resizing and saving images.


In [6]:
import os

# Check if the image exists at this path
image_path = "/content/drive/MyDrive/G178_2 -1080.BMP"
print("Exists:", os.path.exists(image_path))


Exists: True


Now i am going to run 1000 iterations for all 3 interpolations

In [7]:
%%writefile resize_image_1000.cpp
#include <opencv2/opencv.hpp>
#include <iostream>
#include <chrono>

int main() {
    std::string input_path = "/content/drive/MyDrive/G178_2 -1080.BMP";
    cv::Mat src = cv::imread(input_path);

    if (src.empty()) {
        std::cerr << "Error: Could not load image!" << std::endl;
        return -1;
    }

    cv::Size targetSize(src.cols / 2, src.rows / 2);
    cv::Mat resized;

    // INTER_NEAREST
    auto start_nearest = std::chrono::high_resolution_clock::now();
    for (int i = 0; i < 1000; ++i) {
        cv::resize(src, resized, targetSize, 0, 0, cv::INTER_NEAREST);
    }
    auto end_nearest = std::chrono::high_resolution_clock::now();
    auto duration_nearest = std::chrono::duration_cast<std::chrono::milliseconds>(end_nearest - start_nearest).count();
    std::cout << "Time taken for 1000 iterations using INTER_NEAREST: " << duration_nearest << " ms" << std::endl;

    // INTER_LINEAR
    auto start_linear = std::chrono::high_resolution_clock::now();
    for (int i = 0; i < 1000; ++i) {
        cv::resize(src, resized, targetSize, 0, 0, cv::INTER_LINEAR);
    }
    auto end_linear = std::chrono::high_resolution_clock::now();
    auto duration_linear = std::chrono::duration_cast<std::chrono::milliseconds>(end_linear - start_linear).count();
    std::cout << "Time taken for 1000 iterations using INTER_LINEAR: " << duration_linear << " ms" << std::endl;

    // INTER_CUBIC
    auto start_cubic = std::chrono::high_resolution_clock::now();
    for (int i = 0; i < 1000; ++i) {
        cv::resize(src, resized, targetSize, 0, 0, cv::INTER_CUBIC);
    }
    auto end_cubic = std::chrono::high_resolution_clock::now();
    auto duration_cubic = std::chrono::duration_cast<std::chrono::milliseconds>(end_cubic - start_cubic).count();
    std::cout << "Time taken for 1000 iterations using INTER_CUBIC: " << duration_cubic << " ms" << std::endl;

    return 0;
}


Writing resize_image_1000.cpp


In [8]:
!g++ resize_image_1000.cpp -o resize_image_1000 `pkg-config --cflags --libs opencv4`

In [9]:
!./resize_image_1000

Time taken for 1000 iterations using INTER_NEAREST: 828 ms
Time taken for 1000 iterations using INTER_LINEAR: 1207 ms
Time taken for 1000 iterations using INTER_CUBIC: 11732 ms


In [10]:
%%writefile resize_image_custom2.cpp
#include <opencv2/opencv.hpp>
#include <iostream>
#include <chrono>
#include <cmath>

enum InterpolationMethod {
    INTER_NEAREST = 0,
    INTER_LINEAR = 1,
    INTER_CUBIC = 2
};

inline int clamp(int val, int max) {
    return (val < 0) ? 0 : (val > max) ? max : val;
}

void myResize(const cv::Mat& src, cv::Mat& dst, cv::Size dsize, int interpolation) {
    int src_width = src.cols;
    int src_height = src.rows;
    int dst_width = dsize.width;
    int dst_height = dsize.height;

    dst.create(dst_height, dst_width, src.type());

    double scale_x = static_cast<double>(src_width) / dst_width;
    double scale_y = static_cast<double>(src_height) / dst_height;

    for (int y = 0; y < dst_height; ++y) {
        for (int x = 0; x < dst_width; ++x) {
            double gx = (x + 0.5) * scale_x - 0.5;
            double gy = (y + 0.5) * scale_y - 0.5;

            if (interpolation == INTER_NEAREST) {
                int ix = clamp(static_cast<int>(round(gx)), src_width - 1);
                int iy = clamp(static_cast<int>(round(gy)), src_height - 1);

                for (int c = 0; c < src.channels(); ++c) {
                    dst.at<cv::Vec3b>(y, x)[c] = src.at<cv::Vec3b>(iy, ix)[c];
                }
            }
            else if (interpolation == INTER_LINEAR || interpolation == INTER_CUBIC) {
                // For INTER_CUBIC
                int ix = static_cast<int>(floor(gx));
                int iy = static_cast<int>(floor(gy));
                double dx = gx - ix;
                double dy = gy - iy;

                for (int c = 0; c < src.channels(); ++c) {
                    uchar p00 = src.at<cv::Vec3b>(clamp(iy, src_height - 1), clamp(ix, src_width - 1))[c];
                    uchar p01 = src.at<cv::Vec3b>(clamp(iy, src_height - 1), clamp(ix + 1, src_width - 1))[c];
                    uchar p10 = src.at<cv::Vec3b>(clamp(iy + 1, src_height - 1), clamp(ix, src_width - 1))[c];
                    uchar p11 = src.at<cv::Vec3b>(clamp(iy + 1, src_height - 1), clamp(ix + 1, src_width - 1))[c];

                    double val = (1 - dx) * (1 - dy) * p00 +
                                 dx * (1 - dy) * p01 +
                                 (1 - dx) * dy * p10 +
                                 dx * dy * p11;

                    dst.at<cv::Vec3b>(y, x)[c] = static_cast<uchar>(val);
                }
            }
        }
    }
}

bool compareImagesWithTolerance(const cv::Mat& img1, const cv::Mat& img2, int tol = 2) {
    if (img1.size() != img2.size() || img1.type() != img2.type())
        return false;

    cv::Mat diff;
    cv::absdiff(img1, img2, diff);
    for (int y = 0; y < diff.rows; ++y) {
        for (int x = 0; x < diff.cols; ++x) {
            cv::Vec3b pixel = diff.at<cv::Vec3b>(y, x);
            if (pixel[0] > tol || pixel[1] > tol || pixel[2] > tol)
                return false;
        }
    }
    return true;
}

int main() {
    std::string input_path = "/content/drive/MyDrive/G178_2 -1080.BMP";
    cv::Mat src = cv::imread(input_path);
    if (src.empty()) {
        std::cerr << "Error: Could not load image!" << std::endl;
        return -1;
    }

    cv::Size targetSize(src.cols / 2, src.rows / 2);

    cv::Mat cv_resized, my_resized;

    for (int interp : {INTER_NEAREST, INTER_LINEAR, INTER_CUBIC}) {
        // Warm up
        cv::resize(src, cv_resized, targetSize, 0, 0, interp);
        myResize(src, my_resized, targetSize, interp);

        // Measure OpenCV resize timing
        auto start_cv = std::chrono::high_resolution_clock::now();
        for (int i = 0; i < 1000; ++i) {
            cv::resize(src, cv_resized, targetSize, 0, 0, interp);
        }
        auto end_cv = std::chrono::high_resolution_clock::now();
        auto cv_time = std::chrono::duration_cast<std::chrono::milliseconds>(end_cv - start_cv).count();

        // Measure custom resize timing
        auto start_my = std::chrono::high_resolution_clock::now();
        for (int i = 0; i < 1000; ++i) {
            myResize(src, my_resized, targetSize, interp);
        }
        auto end_my = std::chrono::high_resolution_clock::now();
        auto my_time = std::chrono::duration_cast<std::chrono::milliseconds>(end_my - start_my).count();

        // Compare results with tolerance
        bool identical = compareImagesWithTolerance(cv_resized, my_resized, 2);

        std::cout << "Interpolation method: " << interp << " (0=Nearest,1=Linear,2=Cubic)" << std::endl;
        std::cout << "OpenCV resize time (1000 iters): " << cv_time << " ms" << std::endl;
        std::cout << "Custom resize time (1000 iters): " << my_time << " ms" << std::endl;
        std::cout << "Outputs similar (tolerance=2)? " << (identical ? "YES" : "NO") << std::endl;
        std::cout << "-------------------------------------------" << std::endl;
    }

    return 0;
}


Writing resize_image_custom2.cpp


In [11]:
!g++ resize_image_custom2.cpp -o resize_image_custom2 `pkg-config --cflags --libs opencv4`

In [12]:
!./resize_image_custom2

Interpolation method: 0 (0=Nearest,1=Linear,2=Cubic)
OpenCV resize time (1000 iters): 832 ms
Custom resize time (1000 iters): 38201 ms
Outputs similar (tolerance=2)? NO
-------------------------------------------
Interpolation method: 1 (0=Nearest,1=Linear,2=Cubic)
OpenCV resize time (1000 iters): 1987 ms
Custom resize time (1000 iters): 122052 ms
Outputs similar (tolerance=2)? YES
-------------------------------------------
Interpolation method: 2 (0=Nearest,1=Linear,2=Cubic)
OpenCV resize time (1000 iters): 11735 ms
Custom resize time (1000 iters): 124535 ms
Outputs similar (tolerance=2)? NO
-------------------------------------------
